* EDA3에서 저장한 info데이터를 가져온다.
* 리스트 형태로 되어있는 칼럼을 풀어서 하나의 df로 합쳐준다.
* 최종적으로 각 행은 하나의 match 데이터를 가진 통합데이터를 만들고 저장한다.

In [1]:
import numpy as np
import pandas as pd
import requests
import json
from tqdm import tnrange
import time
from sklearn.model_selection import train_test_split

In [2]:
api_key = 'RGAPI-eabfcbf1-25a3-4971-be6c-036dadfe7f42'

In [20]:
data = pd.read_pickle('data/saving_data/match_info.pkl')
data

,gameCreation,gameDuration,gameId,gameMode,gameType,gameVersion,mapId,participantIdentities,participants,platformId,queueId,seasonId,status,teams
0,1588569216123,1475,4353346263,CLASSIC,MATCHED_GAME,10.9.318.6057,11,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,420,13,NaN,"[{'teamId': 100, 'win': 'Win', 'firstBlood': T..."
0,1589030702498,1404,4367199595,CLASSIC,MATCHED_GAME,10.9.318.6057,11,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,420,13,NaN,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ..."
0,1588514577662,1722,4352098588,CLASSIC,MATCHED_GAME,10.9.318.6057,11,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,420,13,NaN,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ..."
0,1588745022341,965,4358736984,CLASSIC,MATCHED_GAME,10.9.318.6057,11,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,420,13,NaN,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ..."
0,1588837514402,812,4361173967,CLASSIC,MATCHED_GAME,10.9.318.6057,11,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,420,13,NaN,"[{'teamId': 100, 'win': 'Win', 'firstBlood': T..."
0,1589372097554,1712,4377039377,CLASSIC,MATCHED_GAME,10.10.320.3039,11,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,420,13,NaN,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ..."
0,1588139841997,1369,4339975651,CLASSIC,MATCHED_GAME,10.9.318.6057,11,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,420,13,NaN,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ..."
0,1589425548230,1508,4378064003,CLASSIC,MATCHED_GAME,10.10.320.3039,11,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,420,13,NaN,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ..."
0,1589276891396,1325,4374104072,CLASSIC,MATCHED_GAME,10.9.318.6057,11,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,420,13,NaN,"[{'teamId': 100, 'win': 'Win', 'firstBlood': T..."
0,1587381375511,1055,4317268687,CLASSIC,MATCHED_GAME,10.8.317.8137,11,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,420,13,NaN,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ..."


In [21]:
# 모든값이 MaM인 status칼럼 삭제
data = data.drop(['status'], axis=1)
data.head()

,gameCreation,gameDuration,gameId,gameMode,gameType,gameVersion,mapId,participantIdentities,participants,platformId,queueId,seasonId,teams
0,1588569216123,1475,4353346263,CLASSIC,MATCHED_GAME,10.9.318.6057,11,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,420,13,"[{'teamId': 100, 'win': 'Win', 'firstBlood': T..."
0,1589030702498,1404,4367199595,CLASSIC,MATCHED_GAME,10.9.318.6057,11,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,420,13,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ..."
0,1588514577662,1722,4352098588,CLASSIC,MATCHED_GAME,10.9.318.6057,11,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,420,13,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ..."
0,1588745022341,965,4358736984,CLASSIC,MATCHED_GAME,10.9.318.6057,11,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,420,13,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ..."
0,1588837514402,812,4361173967,CLASSIC,MATCHED_GAME,10.9.318.6057,11,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,420,13,"[{'teamId': 100, 'win': 'Win', 'firstBlood': T..."


In [22]:
'''
NaN이 들어있는 행은 삭제시켜 teams와 participants칼럼을 풀어서 하나의 dataframe으로 합칠 수 있게 만든다.
'''
data = data.dropna()

In [24]:
# data.isnull().sum()

gameCreation             0
gameDuration             0
gameId                   0
gameMode                 0
gameType                 0
gameVersion              0
mapId                    0
participantIdentities    0
participants             0
platformId               0
queueId                  0
seasonId                 0
teams                    0
dtype: int64

In [26]:
data

,gameCreation,gameDuration,gameId,gameMode,gameType,gameVersion,mapId,participantIdentities,participants,platformId,queueId,seasonId,teams
0,1588569216123,1475,4353346263,CLASSIC,MATCHED_GAME,10.9.318.6057,11,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,420,13,"[{'teamId': 100, 'win': 'Win', 'firstBlood': T..."
0,1589030702498,1404,4367199595,CLASSIC,MATCHED_GAME,10.9.318.6057,11,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,420,13,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ..."
0,1588514577662,1722,4352098588,CLASSIC,MATCHED_GAME,10.9.318.6057,11,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,420,13,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ..."
0,1588745022341,965,4358736984,CLASSIC,MATCHED_GAME,10.9.318.6057,11,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,420,13,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ..."
0,1588837514402,812,4361173967,CLASSIC,MATCHED_GAME,10.9.318.6057,11,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,420,13,"[{'teamId': 100, 'win': 'Win', 'firstBlood': T..."
0,1589372097554,1712,4377039377,CLASSIC,MATCHED_GAME,10.10.320.3039,11,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,420,13,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ..."
0,1588139841997,1369,4339975651,CLASSIC,MATCHED_GAME,10.9.318.6057,11,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,420,13,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ..."
0,1589425548230,1508,4378064003,CLASSIC,MATCHED_GAME,10.10.320.3039,11,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,420,13,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ..."
0,1589276891396,1325,4374104072,CLASSIC,MATCHED_GAME,10.9.318.6057,11,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,420,13,"[{'teamId': 100, 'win': 'Win', 'firstBlood': T..."
0,1587381375511,1055,4317268687,CLASSIC,MATCHED_GAME,10.8.317.8137,11,"[{'participantId': 1, 'player': {'platformId':...","[{'participantId': 1, 'teamId': 100, 'champion...",KR,420,13,"[{'teamId': 100, 'win': 'Fail', 'firstBlood': ..."


In [72]:
'''
teams 칼럼에 들어있는 데이터를 풀어서 하나의 칼럼을 갖도록 df를 만든다.
두개의 팀이 있으므로 각 팀을 나눠서 df를 만들고 합쳐준다.
100 --> blue side
200 --> red side
'''
# blue side
a_ls = list(data['teams'])

team1_df = pd.DataFrame()
for i in range(len(a_ls)):
    try:
        a_ls[i][0].pop('bans',None)
        team1 = pd.DataFrame(list(a_ls[i][0].values()),index = list(a_ls[i][0].keys())).T
        team1_df = team1_df.append(team1)
    except:
        print(i)
    
team1_df.index = range(len(team1_df))

In [82]:
'''
칼럼의 이름이 blue와 red팀이 같아질수 있으므로 blue팀은 칼럼이름앞에 blue 라고 추가해준다.
'''
blue_columns = ['blue'+s for s in list(team1_df.columns)]
blue_columns

['blueteamId',
 'bluewin',
 'bluefirstBlood',
 'bluefirstTower',
 'bluefirstInhibitor',
 'bluefirstBaron',
 'bluefirstDragon',
 'bluefirstRiftHerald',
 'bluetowerKills',
 'blueinhibitorKills',
 'bluebaronKills',
 'bluedragonKills',
 'bluevilemawKills',
 'blueriftHeraldKills',
 'bluedominionVictoryScore']

In [84]:
team1_df.head()

,teamId,win,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,firstRiftHerald,towerKills,inhibitorKills,baronKills,dragonKills,vilemawKills,riftHeraldKills,dominionVictoryScore
0,100,Win,True,False,True,True,True,False,8,1,1,1,0,1,0
1,100,Fail,False,False,False,False,False,False,1,0,0,2,0,0,0
2,100,Fail,True,True,False,False,True,False,3,0,0,3,0,0,0
3,100,Fail,False,False,False,False,True,False,0,0,0,1,0,0,0
4,100,Win,True,True,True,False,True,True,7,1,0,1,0,1,0


In [85]:
'''
칼럼이름이 바뀐것을 확인
'''
team1_df.columns = blue_columns
team1_df

,blueteamId,bluewin,bluefirstBlood,bluefirstTower,bluefirstInhibitor,bluefirstBaron,bluefirstDragon,bluefirstRiftHerald,bluetowerKills,blueinhibitorKills,bluebaronKills,bluedragonKills,bluevilemawKills,blueriftHeraldKills,bluedominionVictoryScore
0,100,Win,True,False,True,True,True,False,8,1,1,1,0,1,0
1,100,Fail,False,False,False,False,False,False,1,0,0,2,0,0,0
2,100,Fail,True,True,False,False,True,False,3,0,0,3,0,0,0
3,100,Fail,False,False,False,False,True,False,0,0,0,1,0,0,0
4,100,Win,True,True,True,False,True,True,7,1,0,1,0,1,0
5,100,Fail,True,True,False,False,True,True,4,0,0,1,0,2,0
6,100,Fail,False,False,False,False,False,False,2,0,0,0,0,0,0
7,100,Fail,True,False,False,False,False,False,1,0,0,1,0,0,0
8,100,Win,True,True,False,True,True,True,6,0,1,2,0,2,0
9,100,Fail,False,True,False,False,False,True,1,0,0,0,0,1,0


In [86]:
'''
red team
'''
team2_df = pd.DataFrame()
for i in range(len(a_ls)):
    try:
        a_ls[i][1].pop('bans',None)
        team2 = pd.DataFrame(list(a_ls[i][1].values()),index = list(a_ls[i][1].keys())).T
        team2_df = team2_df.append(team2)
    except:
        print(i)
    
team2_df.index = range(len(team2_df))

In [87]:
team2_df.head()

,teamId,win,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,firstRiftHerald,towerKills,inhibitorKills,baronKills,dragonKills,vilemawKills,riftHeraldKills,dominionVictoryScore
0,200,Fail,False,True,False,False,False,True,3,0,0,2,0,1,0
1,200,Win,True,True,True,True,True,True,7,1,1,1,0,1,0
2,200,Win,False,False,True,True,False,True,9,3,1,1,0,2,0
3,200,Win,True,True,True,False,False,True,11,1,0,0,0,1,0
4,200,Fail,False,False,False,False,False,False,0,0,0,0,0,0,0


In [88]:
red_columns = ['red'+s for s in list(team2_df.columns)]
red_columns

['redteamId',
 'redwin',
 'redfirstBlood',
 'redfirstTower',
 'redfirstInhibitor',
 'redfirstBaron',
 'redfirstDragon',
 'redfirstRiftHerald',
 'redtowerKills',
 'redinhibitorKills',
 'redbaronKills',
 'reddragonKills',
 'redvilemawKills',
 'redriftHeraldKills',
 'reddominionVictoryScore']

In [89]:
'''
칼럼이름이 바뀐것을 확인
'''
team2_df.columns = red_columns
team2_df

,redteamId,redwin,redfirstBlood,redfirstTower,redfirstInhibitor,redfirstBaron,redfirstDragon,redfirstRiftHerald,redtowerKills,redinhibitorKills,redbaronKills,reddragonKills,redvilemawKills,redriftHeraldKills,reddominionVictoryScore
0,200,Fail,False,True,False,False,False,True,3,0,0,2,0,1,0
1,200,Win,True,True,True,True,True,True,7,1,1,1,0,1,0
2,200,Win,False,False,True,True,False,True,9,3,1,1,0,2,0
3,200,Win,True,True,True,False,False,True,11,1,0,0,0,1,0
4,200,Fail,False,False,False,False,False,False,0,0,0,0,0,0,0
5,200,Win,False,False,True,True,False,False,9,2,1,3,0,0,0
6,200,Win,True,True,True,True,True,True,9,1,1,3,0,1,0
7,200,Win,False,True,True,True,True,True,9,2,1,3,0,2,0
8,200,Fail,False,False,False,False,False,False,0,0,0,0,0,0,0
9,200,Win,True,False,True,False,True,False,7,1,0,2,0,1,0


In [90]:
team2_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9987 entries, 0 to 9986
Data columns (total 15 columns):
redteamId                  9987 non-null object
redwin                     9987 non-null object
redfirstBlood              9987 non-null object
redfirstTower              9987 non-null object
redfirstInhibitor          9987 non-null object
redfirstBaron              9987 non-null object
redfirstDragon             9987 non-null object
redfirstRiftHerald         9987 non-null object
redtowerKills              9987 non-null object
redinhibitorKills          9987 non-null object
redbaronKills              9987 non-null object
reddragonKills             9987 non-null object
redvilemawKills            9987 non-null object
redriftHeraldKills         9987 non-null object
reddominionVictoryScore    9987 non-null object
dtypes: object(15)
memory usage: 1.1+ MB


In [91]:
'''
각 팀의 info를 담은 dataframe의 길이가 일치하는지 확인해준다.
'''
len(team1_df) == len(team2_df) == len(data)

True

* data에서 teams, participants 그리고 duration 칼럼을 사용할것
* [column info](https://developer.riotgames.com/apis#match-v4/GET_getMatch) 에서 각 내용 확인 가능
* 각팀으로 만든 df와 기존 data를 합쳐준다

In [160]:
'''
팀1과 팀2를 합쳐준 df에 추가로 합쳐줄 gameDuration df 생성
--> 이때 reset_index를 꼭 해줘야한다.
    ---> 안하면 cannot reindex from a duplicate axis 오류남
'''
data_pri = data[['gameDuration']]
data_pri = data_pri.reset_index()
data_pri.drop(['index'],axis=1, inplace=True)
data_pri

,gameDuration
0,1475
1,1404
2,1722
3,965
4,812
5,1712
6,1369
7,1508
8,1325
9,1055


In [147]:
'''
team df와 기존 data에서 사용할 칼럼을 선택해 합쳐준다.
먼저 팀1과 팀2를 합쳐주고
'''
data_team = pd.concat([team1_df,team2_df],axis=1)

In [149]:
data_team

,blueteamId,bluewin,bluefirstBlood,bluefirstTower,bluefirstInhibitor,bluefirstBaron,bluefirstDragon,bluefirstRiftHerald,bluetowerKills,blueinhibitorKills,...,redfirstDragon,redfirstRiftHerald,redtowerKills,redinhibitorKills,redbaronKills,reddragonKills,redvilemawKills,redriftHeraldKills,reddominionVictoryScore,gameDuration
0,100,Win,True,False,True,True,True,False,8,1,...,False,True,3,0,0,2,0,1,0,
1,100,Fail,False,False,False,False,False,False,1,0,...,True,True,7,1,1,1,0,1,0,
2,100,Fail,True,True,False,False,True,False,3,0,...,False,True,9,3,1,1,0,2,0,
3,100,Fail,False,False,False,False,True,False,0,0,...,False,True,11,1,0,0,0,1,0,
4,100,Win,True,True,True,False,True,True,7,1,...,False,False,0,0,0,0,0,0,0,
5,100,Fail,True,True,False,False,True,True,4,0,...,False,False,9,2,1,3,0,0,0,
6,100,Fail,False,False,False,False,False,False,2,0,...,True,True,9,1,1,3,0,1,0,
7,100,Fail,True,False,False,False,False,False,1,0,...,True,True,9,2,1,3,0,2,0,
8,100,Win,True,True,False,True,True,True,6,0,...,False,False,0,0,0,0,0,0,0,
9,100,Fail,False,True,False,False,False,True,1,0,...,True,False,7,1,0,2,0,1,0,


In [161]:
'''
마지막으로 data_pri까지 합쳐줘서 통합된 데이터셋을 만들어냈다.
'''
data_team.update(data_pri)

In [163]:
'''
통합데이터 출력 확인
'''
data_team

,blueteamId,bluewin,bluefirstBlood,bluefirstTower,bluefirstInhibitor,bluefirstBaron,bluefirstDragon,bluefirstRiftHerald,bluetowerKills,blueinhibitorKills,...,redfirstDragon,redfirstRiftHerald,redtowerKills,redinhibitorKills,redbaronKills,reddragonKills,redvilemawKills,redriftHeraldKills,reddominionVictoryScore,gameDuration
0,100,Win,True,False,True,True,True,False,8,1,...,False,True,3,0,0,2,0,1,0,1475
1,100,Fail,False,False,False,False,False,False,1,0,...,True,True,7,1,1,1,0,1,0,1404
2,100,Fail,True,True,False,False,True,False,3,0,...,False,True,9,3,1,1,0,2,0,1722
3,100,Fail,False,False,False,False,True,False,0,0,...,False,True,11,1,0,0,0,1,0,965
4,100,Win,True,True,True,False,True,True,7,1,...,False,False,0,0,0,0,0,0,0,812
5,100,Fail,True,True,False,False,True,True,4,0,...,False,False,9,2,1,3,0,0,0,1712
6,100,Fail,False,False,False,False,False,False,2,0,...,True,True,9,1,1,3,0,1,0,1369
7,100,Fail,True,False,False,False,False,False,1,0,...,True,True,9,2,1,3,0,2,0,1508
8,100,Win,True,True,False,True,True,True,6,0,...,False,False,0,0,0,0,0,0,0,1325
9,100,Fail,False,True,False,False,False,True,1,0,...,True,False,7,1,0,2,0,1,0,1055


In [166]:
'''
최종적으로 통합데이터 NaN값 확인 --> 0 확인 완료
'''
data_team.isnull().sum().sum()

0

In [167]:
'''
통합데이터 저장
'''
data_team.to_pickle('data/saving_data/match_integrated_data.pkl')

In [ ]:
'''
통합데이터 불러와서 출력 확인
'''
loa